# Deepseg Circle

In [ ]:
import os
import pandas as pd

prefix = '<path_to>/fov_segmentation'

df = pd.read_pickle(os.path.join(prefix, 'light_log.pkl'))
test_df = df[df.test == True]
test_df

In [ ]:
import cv2
import numpy as np
from kornia import image_to_tensor, tensor_to_image
from lightning_modules import ImageSegmentationModule

from endoscopy import endoscopy
from utils import load_yaml

device = 'cuda' # 'cpu' for cpu exection

# load model with default values
checkpoint_prefix = 'checkpoints/resnet_34'
checkpoint = 'epoch=35-step=1079.ckpt'

config = load_yaml(os.path.join(prefix, checkpoint_prefix, 'config.yml'))

model = ImageSegmentationModule(**config['model'])
model = model.load_from_checkpoint(os.path.join(prefix, checkpoint_prefix, checkpoint), **config['model'])
model = model.eval()
model.freeze()
model = model.to(device)

for idx, row in test_df.iterrows():
    # load and prepare data
    img = np.load(os.path.join(prefix, row.image['folder'], row.image['file']))
    seg_gt = np.load(os.path.join(prefix, row.segmentation['folder'], row.segmentation['file']))

    # perform segmentation
    img = image_to_tensor(img, keepdim=False)
    img = img.to(model.device)
    img = img.float()/255.
    seg = model(img)

    # to image
    img = tensor_to_image(img, keepdim=False)
    seg = tensor_to_image(seg, keepdim=False)

    # detect circle
    center, radius = endoscopy.ransacBoundaryCircle(seg*255, th=10, n_pts=100, n_iter=1, fit='numeric')
    center, radius = center.astype(int), int(radius)

    # show results
    cv2.circle(img, (center[1], center[0]), radius, (0,255,255), 2)  

    cv2.imshow('Segmentation', seg)
    cv2.imshow('Endoscopic View', img)
    cv2.waitKey()
